<a href="https://colab.research.google.com/github/FeraMaks/test/blob/main/NN_ShaftTraectory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd

from google.colab import drive

In [2]:
#device config
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


Hyper parameters

#Создание датасета

In [4]:
xy = pd.read_excel(open('/content/drive/MyDrive/work/100.xlsx', 'rb'))
xy = pd.DataFrame(xy)
xynp = pd.DataFrame.to_numpy(xy)
xynp = np.rot90(xynp)
xynp.shape
xynp1 = np.concatenate((xynp[0,:2000], xynp[1,:2000]))
xynp2 = np.concatenate((xynp[0,2000:4000], xynp[1,2000:4000]))
xynp3 = np.concatenate((xynp[0,4000:6000], xynp[1,4000:6000]))
xynp4 = np.concatenate((xynp[0,6000:8000], xynp[1,6000:8000]))
xynp5 = np.concatenate((xynp[0,8000:10000], xynp[1,8000:10000]))
xynp6 = np.concatenate((xynp[0,10000:12000], xynp[1,10000:12000]))
xynp7 = np.concatenate((xynp[0,12000:14000], xynp[1,12000:14000]))
xynp8 = np.concatenate((xynp[0,14000:16000], xynp[1,14000:16000]))
xynp9 = np.concatenate((xynp[0,16000:18000], xynp[1,16000:18000]))
xynp10 = np.concatenate((xynp[0,18000:20000], xynp[1,18000:20000]))

xynp11 = np.concatenate((xynp[2,:2000], xynp[3,:2000]))
xynp12 = np.concatenate((xynp[2,2000:4000], xynp[3,2000:4000]))
xynp13 = np.concatenate((xynp[2,4000:6000], xynp[3,4000:6000]))
xynp14 = np.concatenate((xynp[2,6000:8000], xynp[3,6000:8000]))
xynp15 = np.concatenate((xynp[2,8000:10000], xynp[3,8000:10000]))
xynp16 = np.concatenate((xynp[2,10000:12000], xynp[3,10000:12000]))
xynp17 = np.concatenate((xynp[2,12000:14000], xynp[3,12000:14000]))
xynp18 = np.concatenate((xynp[2,14000:16000], xynp[3,14000:16000]))
xynp19 = np.concatenate((xynp[2,16000:18000], xynp[3,16000:18000]))
xynp20 = np.concatenate((xynp[2,18000:20000], xynp[3,18000:20000]))

xynp21 = np.concatenate((xynp[4,:2000], xynp[5,:2000]))
xynp22 = np.concatenate((xynp[4,2000:4000], xynp[5,2000:4000]))
xynp23 = np.concatenate((xynp[4,4000:6000], xynp[5,4000:6000]))
xynp24 = np.concatenate((xynp[4,6000:8000], xynp[5,6000:8000]))
xynp25 = np.concatenate((xynp[4,8000:10000], xynp[5,8000:10000]))
xynp26 = np.concatenate((xynp[4,10000:12000], xynp[5,10000:12000]))
xynp27 = np.concatenate((xynp[4,12000:14000], xynp[5,12000:14000]))
xynp28 = np.concatenate((xynp[4,14000:16000], xynp[5,14000:16000]))
xynp29 = np.concatenate((xynp[4,16000:18000], xynp[5,16000:18000]))
xynp30 = np.concatenate((xynp[4,18000:20000], xynp[5,18000:20000]))

x = np.vstack((xynp1,xynp2,xynp3,xynp4,xynp5,xynp6,xynp7,xynp11,xynp12,xynp13,xynp14,xynp15,xynp16,xynp17,xynp21,xynp22,xynp23,xynp24,xynp25,xynp26,xynp27,xynp8,xynp9,xynp10,xynp18,xynp19,xynp20,xynp28,xynp29,xynp30))
x.shape

(30, 4000)

In [5]:
x[0,:]

array([-182.154261, -182.813766, -183.424184, ...,  -46.86475 ,
        -44.582413,  -42.288928])

In [6]:
y1 = np.ones((20,1))
y2 = np.ones((10,1))
y2 = y2*2
y = np.concatenate((y1,y2))
y.shape

(30, 1)

In [7]:
yx = np.concatenate((y,x), axis=1)
yx

array([[   1.      , -182.154261, -182.813766, ...,  -46.86475 ,
         -44.582413,  -42.288928],
       [   1.      , -173.012446, -173.965359, ...,  -75.228733,
         -72.970316,  -70.706292],
       [   1.      , -160.957252, -161.887317, ..., -101.26894 ,
         -99.136526,  -96.993631],
       ...,
       [   2.      ,   39.184434,   32.518965, ..., -414.807569,
        -417.559574, -420.19372 ],
       [   2.      ,  111.369368,  104.954632, ..., -372.087845,
        -376.125338, -380.059501],
       [   2.      ,  187.787242,  181.505511, ..., -317.671315,
        -322.631982, -327.522761]])

In [8]:
class ShaftDataset(Dataset):
  def __init__(self):
    ds = yx
    self.x = torch.from_numpy(ds[:, 1:])
    self.y = torch.from_numpy(ds[:, 0])
    self.n = ds.shape[0]

  def __getitem__(self, index):
    return self.x[index], self.y[index]
  
  def __len__(self):
    return self.n

In [9]:
dataset = ShaftDataset()
first_data = dataset[0]
print(first_data)

(tensor([-182.1543, -182.8138, -183.4242,  ...,  -46.8648,  -44.5824,
         -42.2889], dtype=torch.float64), tensor(1., dtype=torch.float64))


Hyper parameters

In [10]:
input_size = x.shape[0]*x.shape[1]
hidden_size = 100
num_classes = 2
num_epochs = 10
batch_size = 4000
lr = 0.001 # learning rate

In [18]:
train_dataset = x[:21,:]
test_dataset = x[21:,:]
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
train_dataset = train_dataset.torch.transforms.ToTensor()

AttributeError: ignored

In [12]:
examples = iter(train_loader)
samples, labels = x, y
print(samples.shape, labels.shape)

(30, 4000) (30, 1)


#Создание нейронной сети

In [13]:
class NeuralNet(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(NeuralNet, self).__init__()
    self.l1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.l2 = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    out = self.l1(x)
    out = self.relu(out)
    out = self.l2(out)
    return out


In [14]:
model = NeuralNet(input_size, hidden_size, num_classes)

Loss and optimizer

In [15]:
L = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

#Training loop

In [16]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_loader):
    images = images.reshape(-1, x.shape[0]*x.shape[1]).to(device)
    labels = labels.to(device)

    #forward path
    outputs = model(images)
    loss = L(outputs, labels)

    #backward path
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (i+1) % 100 == 0:
       print(f'epoch {epoch+1} / {num_epochs}, step {i+1}/{n_total_steps}, loss = {loss.item():.4f}')


ValueError: ignored

#Test

In [ ]:
with torch.no_grad():
  n_correct = 0
  n_samples = 0
  for images, labels in test_loader:
    images = images.reshape(-1, x.shape[0]*x.shape[1]).to(device)
    labels = labels.to(device)
    outputs = model(images)

    #value, index
    _, predictions = torch.max(outputs, 1)
    n_samples += labels.shape[0]
    n_correct += (predictions == labels).sum().item()
  acc = 100.0 * n_correct / n_samples
  print(f'accuracy = {acc}')